09_04. 워드투벡터(Word2Vec)

In [8]:
import os, sys 
from google.colab import drive 
drive.mount('/content/drive') 

import sys
sys.path.append('/content/drive/MyDrive/haicon') # 업로드 할 경로
print(sys.path)

Mounted at /content/drive
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/haicon']


In [1]:
import nltk
nltk.download('punkt')

import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# 데이터 불러오기
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fde6d985310>)

훈련 데이터 전처리하기

In [3]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 저자의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

In [4]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [5]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


Word2Vec 훈련시키기

In [6]:
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

#size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBOW, 1은 Skip-gram.

In [7]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8411566615104675), ('guy', 0.8111566305160522), ('girl', 0.7415400147438049), ('boy', 0.7406051754951477), ('lady', 0.7363632917404175), ('soldier', 0.7256036400794983), ('gentleman', 0.707797646522522), ('kid', 0.6844278573989868), ('friend', 0.6616865992546082), ('surgeon', 0.6549986600875854)]


Word2Vec 모델 저장하고 로드하기

In [9]:
model.wv.save_word2vec_format('/content/drive/MyDrive/study/save/eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/study/save/eng_w2v") # 모델 로드

In [10]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8411566615104675), ('guy', 0.8111566305160522), ('girl', 0.7415400147438049), ('boy', 0.7406051754951477), ('lady', 0.7363632917404175), ('soldier', 0.7256036400794983), ('gentleman', 0.707797646522522), ('kid', 0.6844278573989868), ('friend', 0.6616865992546082), ('surgeon', 0.6549986600875854)]
